In [ ]:
!wget https://cloud.isir.upmc.fr/owncloud/index.php/s/f77sSKkyKtek3JH/download

--2021-01-01 15:15:27--  https://cloud.isir.upmc.fr/owncloud/index.php/s/f77sSKkyKtek3JH/download
Resolving cloud.isir.upmc.fr (cloud.isir.upmc.fr)... 134.157.19.7
Connecting to cloud.isir.upmc.fr (cloud.isir.upmc.fr)|134.157.19.7|:443... connected.
HTTP request sent, awaiting response... 503 Service Unavailable
2021-01-01 15:15:28 ERROR 503: Service Unavailable.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg as LA

In [ ]:
import zipfile
with zipfile.ZipFile("download", 'r') as zip_ref:
    zip_ref.extractall("./")
!ls

FileNotFoundError: ignored

In [ ]:
!tail 300w_train_images.txt

In [ ]:
!tail 300w_train_landmarks.txt

In [ ]:
images = open('300w_train_images.txt', 'r').read().splitlines()
print(images[:10])

In [ ]:
landmarks = open('300w_train_landmarks.txt', 'r').read().splitlines()
print(landmarks[:10])

In [ ]:
import matplotlib.image as mpimg
import random

def get_x_y(index):
    landmark_file = open(landmarks[index], 'r').read()
    coordinates = landmark_file.splitlines()
    x = []
    y = []
    for i in coordinates:
        z = i.split(' ')
        x.append(float(z[0]))
        y.append(float(z[1]))
    return x, y

def plot_points(index):
    image = mpimg.imread(images[index])
    x, y = get_x_y(index)
    plt.figure()
    plt.imshow(image, cmap='gray', origin='upper')
    plt.plot(x,y,'o')
    plt.show()

for i in range(10):
    index = random.randint(0, len(landmarks))
    plot_points(index)

In [ ]:
from skimage.transform import resize
import numpy as np

def transform_image(index, plot=False):
    image = mpimg.imread(images[index])
    x, y = get_x_y(index)
    width = image.shape[0]
    height = image.shape[1]

    x_min = int(min(x)-(0.15*min(x)))
    x_max = int(max(x) +  (0.15*max(x) ) )
    y_min = int(min(y)-(0.15*min(y)))
    y_max = int(max(y) +  (0.15*max(y)          )  )

    new_image = image[y_min:y_max, x_min:x_max]
    new_image = resize(new_image, (128, 128))
    x = np.array(x)
    y = np.array(y)
    x =  (x -x_min )
    y =  (y -y_min)
    
    x = x/(x_max-x_min)*128
    y = y/(y_max-y_min)*128
    if plot:
        plt.figure()
        plt.imshow(new_image, cmap='gray', origin='upper')
        plt.plot(x,y,'o')
        plt.show()
    return x, y
plot_points(10)
transform_image(10, plot=True)

In [ ]:
def moyenne_point_caracteristiques(plot=False):
    xmat = np.zeros((68, len(landmarks)  ))
    ymat = np.zeros((68, len(landmarks)   ))
    for i in range(len(landmarks)):
        x, y = transform_image(i)
        xmat[:,i] = np.array(x)
        ymat[:,i] = np.array(y)
    x_moy, y_moy = xmat.mean(axis=1), ymat.mean(axis=1)
    plt.figure()
    plt.plot(x_moy, y_moy,'o')
    plt.show()
    return x_moy, y_moy

x_moy, y_moy = moyenne_point_caracteristiques(plot=True)

In [ ]:
import cv2

def random_perturbation(x, y, plot=False):
    translation = np.random.uniform(low=-20, high=20, size=(10,2))
    scale = np.random.uniform(low=0.8, high=1.2, size=10)
    n_data = np.array(data).repeat(10,axis=0).reshape((68,10,2)) + translation
    
    for i in range(scale.shape[0]):
        n_data[:,i] = int(n_data[:,i] * scale[i])
        
    return np.dstack(n_data.T)  

def get_keyspoints(x, y):
    return np.array([cv2.KeyPoint(x[i],y[i],20) for i in range(len(x))])

def sift_descriptor(image, x, y):
    sift = cv2.xfeatures2d.SIFT_create()
    _, descriptor = sift.compute(image,keysPoint)
    return descriptor.reshape(descriptor.shape[0] * descriptor.shape[1])

def sift_matrix(x, y):
    x_moy, y_moy = moyenne_point_caracteristiques()
    key_points = get_keyspoints(x_moy, y_moy)
    return np.array([compute_sift_descriptors(i[0],key_points) for i in data]) 


random_x = []
random_y = []
for i in range(10): 
    x_rand, y_rand = random_perturbation(x, y, plot=True)
    random_x.append(x_rand)
    random_y.append(y_rand)

In [ ]:
from sklearn.decomposition import PCA

x, y = 


for i in range(len(landmarks)):
        x, y = transform_image(i)
        xmat[:,i] = np.array(x)
        ymat[:,i] = np.array(y)
matrix = sift_matrix(h_data)


pca = PCA(0.98)
X = pca.fit(matrix)
X = pca.transform(matrix)

projection_matrix = pca.components_

In [ ]:
Ytilde = np.column_stack((X,np.ones(X.shape[0])))
mean = mean_caracteristics_points(h_data)
deltaTilde = np.array(list(map(lambda x : np.array(x[1]).flatten() - mean.flatten() , h_data)))
Ro = np.linalg.inv(Ytilde.T.dot(Ytilde)).dot(Ytilde.T).dot(deltaTilde)
delta = Ytilde.dot(Ro)
delta_reshape = delta.reshape((delta.shape[0],68,2))

In [ ]:
for j in range(5):

    plt.figure()
    plt.imshow(h_data[j][0])
    for k,i in enumerate(mean)  :
        plt.plot(int(h_data[j][1][k][0]), int(h_data[j][1][k][1]), 'r+')
        #plt.plot(int(i[0]), int(i[1]), 'b+')
        plt.plot(int(delta_reshape[j][k][0]) + int(i[0]),int(delta_reshape[j][k][1]) + int(i[1]), 'g+')
        #print(int(i[0]))

plt.show()